In [1]:
!pip install tensorflow-datasets

  Using cached https://files.pythonhosted.org/packages/e2/14/900746f9b8e1ed8ea93661f1a62b3937a5a25858d550ece38de8ea579f2d/tensorflow_datasets-1.3.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/5a/81/221d09d90176fd90aed4b530e31b8fedf207385767c06d1d46c550c5e418/promise-2.2.1.tar.gz
  Using cached https://files.pythonhosted.org/packages/4e/c2/e4ed82a725c9f8160a0ed73f0511773be9f76343def86f6f47121f0e8430/tensorflow_metadata-0.15.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/c7/11/345f3173809cea7f1a193bfbf02403fff250a3360e0e118a1630985e547d/dill-0.3.1.1.tar.gz
  Using cached https://files.pythonhosted.org/packages/eb/ee/e59e74ecac678a14d6abefb9054f0bbcb318a6452a30df3776f133886d7d/googleapis-common-protos-1.6.0.tar.gz
  Created wheel for promise: filename=promise-2.2.1-cp36-none-any.whl size=21294 sha256=656f4e511d0512241fee056a98cdae07e8d9893269720c83914e3551bdbdedfd
  Stored in directory: C:\Users\Affinity\AppData\Local\pip\Cache\wh

In [2]:
!python -m pip install --upgrade pip

Requirement already up-to-date: pip in c:\users\affinity\anaconda3\lib\site-packages (19.3.1)


In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
# from keras.models import load_model
from datetime import datetime
#tf.enable_eager_execution()

In [4]:
train_data = tfds.load(name='cifar10:3.*.*', split='train[:80%]', as_supervised=True)
valid_data = tfds.load(name='cifar10:3.*.*', split='train[80%:]', as_supervised=True)
test_data = tfds.load(name='cifar10:3.*.*', split='test', as_supervised=True)

In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same', input_shape=(32, 32, 3)),
    tf.keras.layers.BatchNormalization(),  # Gradient Vanishing / Gradient Exploding 이 일어나지 않도록 하는 아이디어 중의 하나이다
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),  # Gradient Vanishing / Gradient Exploding 이 일어나지 않도록 하는 아이디어 중의 하나이다
    tf.keras.layers.MaxPooling2D(),    
    tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),  # Gradient Vanishing / Gradient Exploding 이 일어나지 않도록 하는 아이디어 중의 하나이다
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='softmax')
    
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 128)       3584      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 128)       512       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 64)        73792     
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 32)          1

In [6]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

train_data = train_data.map(scale).shuffle(50000).batch(64) # 
valid_data = valid_data.map(scale).batch(64)
test_data = test_data.map(scale).batch(64)

In [8]:
dt = datetime.now().strftime('%Y%m%d-%H%M%S')

cp_callback = tf.keras.callbacks.ModelCheckpoint(   
    'checkpoints/{}.hdf5'.format(dt),
    monitor='val_acc',
    save_best_only=True)

In [9]:
class MyCallback(tf.keras.callbacks.Callback):

    def on_epoch_start(self, epoch, logs=None):
        print('Epoch {} begins at {}: {}'.format(epoch, datetime.now().time(), logs))

    def on_epoch_end(self, epoch, logs=None):
        print('Epoch {} ends at {}: {}'.format(epoch, datetime.now().time(), logs))

In [10]:
history = model.fit(train_data,
                    epochs=20,
                    validation_data=valid_data,
                    verbose=0,
                    callbacks=[cp_callback, MyCallback()])

Epoch 0 ends at 15:27:00.702932: {'loss': 1.7730145391464234, 'accuracy': 0.414525, 'val_loss': 1.2942685071070483, 'val_accuracy': 0.541}


Epoch 1 ends at 15:27:19.015566: {'loss': 1.2040796027183533, 'accuracy': 0.57425, 'val_loss': 1.084419104324025, 'val_accuracy': 0.62}


Epoch 2 ends at 15:27:38.278823: {'loss': 1.0368134649276732, 'accuracy': 0.634175, 'val_loss': 1.1476710685499154, 'val_accuracy': 0.6053}


Epoch 3 ends at 15:27:57.926151: {'loss': 0.9497518507957459, 'accuracy': 0.6667, 'val_loss': 1.0558139529957133, 'val_accuracy': 0.6291}


Epoch 4 ends at 15:28:17.881233: {'loss': 0.8951839470863342, 'accuracy': 0.68825, 'val_loss': 1.0761034188756518, 'val_accuracy': 0.6158}


...

Epoch 15 ends at 15:32:22.924802: {'loss': 0.60522764878273, 'accuracy': 0.7864, 'val_loss': 0.9581547166891159, 'val_accuracy': 0.6748}


Epoch 16 ends at 15:32:47.640073: {'loss': 0.5950289278507233, 'accuracy': 0.792725, 'val_loss': 0.7620173969845863, 'val_accuracy': 0.7448}


Epoch 17 ends at 15:33:12.096546: {'loss': 0.5818280215024948, 'accuracy': 0.794875, 'val_loss': 0.8222706851306235, 'val_accuracy': 0.7262}


Epoch 18 ends at 15:33:38.440674: {'loss': 0.5633057186841964, 'accuracy': 0.8013, 'val_loss': 0.7671207140205772, 'val_accuracy': 0.7404}


Epoch 19 ends at 15:34:04.878331: {'loss': 0.5551151056289673, 'accuracy': 0.805525, 'val_loss': 0.8681897145168037, 'val_accuracy': 0.7108}


In [11]:
model.evaluate(test_data)

157/157 [==============================].6721 - accuracy: 0.79 - 0s 60ms/step - loss: 0.7232 - accuracy: 0.78 - 0s 53ms/step - loss: 0.7726 - accuracy: 0.77 - 0s 47ms/step - loss: 0.7636 - accuracy: 0.76 - 0s 46ms/step - loss: 0.7837 - accuracy: 0.76 - 0s 45ms/step - loss: 0.7670 - accuracy: 0.76 - 0s 43ms/step - loss: 0.7763 - accuracy: 0.75 - 0s 42ms/step - loss: 0.7897 - accuracy: 0.75 - 0s 41ms/step - loss: 0.7880 - accuracy: 0.75 - 0s 39ms/step - loss: 0.7737 - accuracy: 0.75 - 0s 38ms/step - loss: 0.7608 - accuracy: 0.76 - 0s 37ms/step - loss: 0.7713 - accuracy: 0.75 - 0s 37ms/step - loss: 0.7721 - accuracy: 0.75 - 1s 36ms/step - loss: 0.7755 - accuracy: 0.75 - 1s 36ms/step - loss: 0.7713 - accuracy: 0.75 - 1s 36ms/step - loss: 0.7801 - accuracy: 0.74 - 1s 35ms/step - loss: 0.7950 - accuracy: 0.74 - 1s 36ms/step - loss: 0.7992 - accuracy: 0.74 - 1s 37ms/step - loss: 0.8015 - accuracy: 0.73 - 1s 37ms/step - loss: 0.8029 - accuracy: 0.74 - 1s 36ms/step - loss: 0.7945 - accuracy: 0.

[0.8480432811816028, 0.715]

In [16]:
m = tf.keras.models.load_model('checkpoints/{}.hdf5'.format(dt))

OSError: SavedModel file does not exist at: checkpoints/20191028-152638.hdf5/{saved_model.pbtxt|saved_model.pb}